### Prueba con codigo hecho con chatgpt

In [ ]:
import pandas as pd
import re

# Load the CSV file with junction coordinates
traza_df = pd.read_csv('/dynamoData/TrazaXYZ_5.csv')

# Extract coordinates from the single column
def extract_coordinates(row):
    match = re.search(r'X = (\d+\.\d+)\s+Y = (\d+\.\d+)\s+Z = (\d+\.\d+)', row)
    if match:
        return float(match.group(1)), float(match.group(2)), float(match.group(3))
    return None, None, None

traza_df[['x', 'y', 'z']] = traza_df.iloc[:, 0].apply(lambda x: pd.Series(extract_coordinates(x)))

# Path to the SWMM input file
inp_file_path = '/mnt/data/A-12-CAV-SWMM-Actual_test.inp'
output_file_path = '/mnt/data/modified_A-12-CAV-SWMM-Actual_test.inp'

# Read the SWMM input file
with open(inp_file_path, 'r') as file:
    lines = file.readlines()

# Find the [JUNCTIONS] section
junctions_start = None
for i, line in enumerate(lines):
    if line.startswith('[JUNCTIONS]'):
        junctions_start = i
        break

if junctions_start is None:
    raise ValueError("No [JUNCTIONS] section found in the INP file")

# Find the end of the [JUNCTIONS] section
junctions_end = None
for i in range(junctions_start + 1, len(lines)):
    if lines[i].startswith('['):
        junctions_end = i
        break

if junctions_end is None:
    junctions_end = len(lines)

# Create a new list of lines with the added junctions
new_lines = lines[:junctions_end]

# Add new junctions from the CSV file
for index, row in traza_df.iterrows():
    if None not in (row['x'], row['y'], row['z']):
        junction_id = f"NewJunction{index + 1}"
        x_coord = row['x']
        y_coord = row['y']
        invert_elev = row['z']
        max_depth = 5.0  # Example value, adjust as necessary
        init_depth = 0.0
        sur_depth = 0.0
        aponded = 0.0

        junction_line = f" {junction_id:<16} {invert_elev:<10} {max_depth:<10} {init_depth:<10} {sur_depth:<10} {aponded:<10}\n"
        new_lines.append(junction_line)

# Add the rest of the original lines after the [JUNCTIONS] section
new_lines.extend(lines[junctions_end:])

# Write the new lines to the modified INP file
with open(output_file_path, 'w') as file:
    file.writelines(new_lines)

output_file_path